# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
# Import Dependencies.

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import logging
import os

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from sklearn.model_selection import train_test_split

import pandas as pd
from azureml.widgets import RunDetails

import joblib 

from azureml.core.model import Model

from azureml.pipeline.core import PipelineData, TrainingOutput

from azureml.pipeline.core import Pipeline

import pickle

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.webservice import LocalWebservice
import requests
import json

#ULIZEUZ: Check this dependencies
#
#import csv
#
#from matplotlib import pyplot as plt
#import numpy as np

#from sklearn import datasets
#import pkg_resources
#
#from azureml.pipeline.steps import AutoMLStep



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview

There are some factors that affects Death Event. We use Kaggle's dataset Heart Failure Clinical Records which contains  information like age, sex, blood pressure, smoke, diabetes, ejection fraction, creatinine phosphokinase, serum_creatinine, serum_sodium, time and we have to predict their death_event.

Dataset was uploaded and register as dataset in the workspace


In [14]:
# Create a workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


quick-starts-ws-130424
aml-quickstarts-130424
southcentralus
3ec87f2b-33cf-4aed-b36f-750175813524
Found existing cluster, use it.

Running


AttributeError: 'DataFrame' object has no attribute 'to_cvs'

In [ ]:
# choose a name for experiment
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

experiment



In [ ]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "notebook130424"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# For a more detailed view of current AmlCompute status, use get_status().


In [17]:
#upload Heart Failure Dataset

data = pd.read_csv("heart_failure_clinical_records_dataset.csv")
data.describe()


Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=1 )

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
    
train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)


In [ ]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))
# train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])


In [ ]:
test.to_csv(path_or_buf='testset.csv', index=False)
data_store.upload_files(['testset.csv'])

test_data = TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

We are trying to predict DEATH_EVENT, therefore we need to define the task as classification. Also define Featurization as auto and enable early stopping and the primary metric AUC_weighted

In [18]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'    
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    compute_target = compute_target,
    task = "classification",
    training_data = train_data,
    label_column_name = "DEATH_EVENT",   
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_setting 
)

In [19]:
# TODO: Submit your experiment

# Create Pipeline and AutoMLStep / outputs AutoMLStep, TrainingOutput

metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=data_store,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=data_store,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))


# remote_run = experiment.submit(automl_config, show_output = True)
# RunDetails(remote_run).show()

#Create an AutoMLStep


automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])


Running on remote.
No run_configuration provided, running on notebook130424 with default configuration
Running on remote compute: notebook130424
Parent Run ID: AutoML_d870120d-3997-4d6e-84d6-f7f2d42cbaaa



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
pipeline_run = experiment.submit(pipeline, show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_d870120d-3997-4d6e-84d6-f7f2d42cbaaa',
 'target': 'notebook130424',
 'status': 'Completed',
 'startTimeUtc': '2020-12-14T03:41:49.397147Z',
 'endTimeUtc': '2020-12-14T04:03:44.179945Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook130424',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-1","subscription_id":"3ec87f2b-33cf-4aed-b36f-750175813524","resource_group":"aml-quickstarts-130424","workspace_name":"quick-starts-ws-130424","region":"southcentralus","compute_target":"notebook130424","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":nul

In [ ]:
RunDetails(pipeline_run).show()


In [ ]:
pipeline_run.wait_for_completion()

# ULIZEUZ: Remove
# remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve metrics of all child runs

metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)


In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

# Retrieve and save your best automl model.

best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model


In [ ]:
best_model.steps

# ULIZEUZ: Remove
# best_model, fitted_model = remote_run.get_output()
# print(best_model)
# print(fitted_model)

In [ ]:
#TODO: Save the best model
joblib.dump(best_model, "models/automl_hearth.pkl")


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
description = 'AutoML Best Model'

model2deploy = remote_run.register_model("description")

print(remote_run.model_id)


In [ ]:
model = Model.register(model_path = "./models",
                       model_name = "Best_Model",
                       description = "Best model trained with AutoML",
                       workspace = ws)


In [ ]:
# Create the environment
myenv = Environment(name="myenv")
# conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
myenv.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
myenv.python.conda_dependencies.add_pip_package("joblib")

myenv.docker.base_image = None
myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
myenv.inferencing_stack_version = "latest"

# conda_dep.add_conda_package("tensorflow")
# conda_dep.add_conda_package("numpy")
# conda_dep.add_conda_package("scikit-learn")
# You must list azureml-defaults as a pip dependency
# conda_dep.add_pip_package("azureml-defaults")
# conda_dep.add_pip_package("keras")
# conda_dep.add_pip_package("gensim")

# Adds dependencies to PythonSection of myenv
# myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
# Define Deployment

# ULIZEUZ: Check Docker
deployment_config = LocalWebservice.deploy_configuration(#port=6789) 

# model = Model(ws, name='sentiment')
service = Model.deploy(ws, 'myservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = 'scoring uri for your service'
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'text': 'Today is a great day!'})

response = requests.post(scoring_uri, data=test_data, headers=headers)


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(response.status_code)
print(response.elapsed)
print(response.json())

service.delete()